# Amazon Kinesis Data Firehose

Amazon Kinesis Data Firehose is a fully managed service for delivering real-time streaming data to destinations such as Amazon Simple Storage Service (Amazon S3), Amazon Redshift, Amazon Elasticsearch Service (Amazon ES), Splunk, and any custom HTTP endpoint. 

For Amazon S3 destinations, streaming data is delivered to your S3 bucket. If data transformation is enabled, you can optionally back up source data to another Amazon S3 bucket.

![](img/kinesis_firehose_s3_docs.png)


In [ ]:
import boto3
import sagemaker
import pandas as pd
import json

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)
firehose = boto3.Session().client(service_name='firehose', region_name=region)

In [ ]:
%store -r firehose_name

In [ ]:
print(firehose_name)

In [ ]:
%store -r iam_role_kinesis_arn

In [ ]:
print(iam_role_kinesis_arn)

# Create a Kinesis Data Firehose Delivery Stream
_This may take 1-2 minutes.  Please be patient._

In [ ]:
from botocore.exceptions import ClientError

try: 
    response = firehose.create_delivery_stream(
        DeliveryStreamName=firehose_name,
        DeliveryStreamType='DirectPut',
        S3DestinationConfiguration={
            'RoleARN': iam_role_kinesis_arn,
            'BucketARN': 'arn:aws:s3:::{}'.format(bucket),
            'Prefix': 'kinesis-data-firehose',        
        }
    )
    print('Delivery stream {} successfully created.'.format(firehose_name))
    print(json.dumps(response, indent=4, sort_keys=True, default=str))
except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceInUseException':
        print('Delivery stream {} already exists.'.format(firehose_name))
    else:
        print('Unexpected error: %s' % e)
    

In [ ]:
import time

status = ''
while status != 'ACTIVE':    
    r = firehose.describe_delivery_stream(DeliveryStreamName=firehose_name)
    description = r.get('DeliveryStreamDescription')
    status = description.get('DeliveryStreamStatus')
    time.sleep(5)
    
print('Delivery Stream {} is active'.format(firehose_name))

In [ ]:
r = firehose.describe_delivery_stream(DeliveryStreamName=firehose_name)

status = description.get('DeliveryStreamStatus')
print(status)

print()

description = r.get('DeliveryStreamDescription')
print(json.dumps(description, indent=4, sort_keys=True, default=str))

In [ ]:
firehose_arn = r['DeliveryStreamDescription']['DeliveryStreamARN']
print(firehose_arn)

# Store Variables for the Next Notebooks

In [ ]:
%store firehose_arn

In [ ]:
%store

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();

References: 

* https://github.com/aws-samples/aws-ml-data-lake-workshop
* https://aws.amazon.com/blogs/big-data/snakes-in-the-stream-feeding-and-eating-amazon-kinesis-streams-with-python/
